In [73]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from keras.preprocessing.text import Tokenizer
import os

In [74]:
df_train=pd.read_csv("complete_dataset.csv")

In [75]:
df_train.tail()

,label,statement,subject,speaker,party_affiliation,barely_true_counts,false_counts,half_true_counts,mostly_true_counts,pants_on_fire_counts,context,extracted_justification
12787,half-true,"For the first time in more than a decade, impo...","energy,oil-spill,trade",barack-obama,democrat,70.0,71.0,160.0,163.0,9.0,a press conference,"In 2009, 17 percent of the U. S. 's oil import..."
12788,mostly-true,Says Donald Trump has bankrupted his companies...,candidates-biography,hillary-clinton,democrat,40.0,29.0,69.0,76.0,7.0,a speech on the economy,"Clinton said, Trump has ""bankrupted his compan..."
12789,true,"John McCain and George Bush have ""absolutely n...",health-care,campaign-defend-america,none,0.0,1.0,0.0,2.0,0.0,a television ad,"""I don't think that there should be a mandate ..."
12790,false,A new poll shows 62 percent support the presid...,health-care,americans-united-change,none,1.0,4.0,4.0,1.0,0.0,an Internet ad.,But the poll doesn't say that. Several days af...
12791,barely-true,No one claims the report vindicating New Jerse...,"candidates-biography,infrastructure",rudy-giuliani,republican,9.0,11.0,10.0,7.0,3.0,"comments on NBC's ""Meet the Press""","Giuliani said ""no one"" called the Mastro repor..."


In [76]:
df_train.shape

(12792, 12)

In [77]:
df_train.dropna(subset = ["statement","extracted_justification"], inplace=True)

In [78]:
df_train.shape

(12691, 12)

In [79]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12691 entries, 0 to 12791
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   label                    12691 non-null  object 
 1   statement                12691 non-null  object 
 2   subject                  12691 non-null  object 
 3   speaker                  12691 non-null  object 
 4   party_affiliation        12691 non-null  object 
 5   barely_true_counts       12691 non-null  float64
 6   false_counts             12691 non-null  float64
 7   half_true_counts         12691 non-null  float64
 8   mostly_true_counts       12691 non-null  float64
 9   pants_on_fire_counts     12691 non-null  float64
 10  context                  12564 non-null  object 
 11  extracted_justification  12691 non-null  object 
dtypes: float64(5), object(7)
memory usage: 1.3+ MB


In [80]:
X_data=df_train["statement"]+df_train["extracted_justification"]

In [81]:
import string
punctuation = string.punctuation

In [82]:
print(punctuation)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [83]:
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords 
from nltk.corpus import sentiwordnet as swn
english_stopwords = set(stopwords.words('english'))
ps = PorterStemmer()

In [84]:
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer
ps=PorterStemmer()

In [85]:
english_stopwords=set(stopwords.words("english"))

In [86]:
STOPWORDS = set(stopwords.words('english'))
from bs4 import BeautifulSoup

In [87]:
#Text Cleaning Functions
def remove_punc(row):
    tokenizer=RegexpTokenizer('[a-zA-Z]+')
    tokens=tokenizer.tokenize(row)
    row=row.lower()
    tokens=row.split() 
    new_tokens=[w for w in tokens if w not in punctuation]
    return new_tokens

In [88]:
X_data=X_data.apply(remove_punc)

In [89]:
X_data

0        [when, did, the, decline, of, coal, start?, it...
1        [hillary, clinton, agrees, with, john, mccain,...
2        [health, care, reform, legislation, is, likely...
3        [the, economic, turnaround, started, at, the, ...
4        [the, chicago, bears, have, had, more, startin...
                               ...                        
12787    [for, the, first, time, in, more, than, a, dec...
12788    [says, donald, trump, has, bankrupted, his, co...
12789    [john, mccain, and, george, bush, have, "absol...
12790    [a, new, poll, shows, 62, percent, support, th...
12791    [no, one, claims, the, report, vindicating, ne...
Length: 12691, dtype: object

In [90]:
additional_words={'com','http','www','stori','wa','hi','ha','pleas'}


In [91]:
total_stopwords=english_stopwords.union(additional_words)

In [92]:
def proc(row):
    text = [ps.stem(word) for word in row]
    text = [w for w in text if w not in total_stopwords]
    text = [w.encode('ascii', 'ignore').decode('ascii') for w in text]
    return text


In [93]:
def rejoin(ro):
    joined_text=(" ".join(ro))
    return joined_text

In [94]:
X_data=X_data.apply(proc)
X_data=X_data.apply(rejoin)

In [95]:
X_data

0        declin coal start? start natur ga took start b...
1        hillari clinton agre john mccain "bi vote give...
2        health care reform legisl like mandat free sex...
3        econom turnaround start end term.crist said ec...
4        chicago bear start quarterback last 10 year to...
                               ...                        
12787    first time decade, import account less half (t...
12788    say donald trump bankrupt compani once, twice ...
12789    john mccain georg bush "absolut plan univers h...
12790    new poll show 62 percent support president' pl...
12791    one claim report vindic new jersey gov. chri c...
Length: 12691, dtype: object

In [96]:
Y=df_train.iloc[:,0]

In [97]:
Y

0          half-true
1        mostly-true
2              false
3          half-true
4               true
            ...     
12787      half-true
12788    mostly-true
12789           true
12790          false
12791    barely-true
Name: label, Length: 12691, dtype: object

In [98]:
from sklearn.preprocessing import LabelEncoder

In [99]:
le=LabelEncoder()

In [100]:
Y=le.fit_transform(Y)

In [101]:
Y

array([2, 3, 1, ..., 5, 1, 0])

In [102]:
0 in Y

True

In [103]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

print('Indexing word vectors.')

embeddings_index = {}
with open('glove.6B.50d.txt',encoding="utf-8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

print('Found %s word vectors.' % len(embeddings_index))


Indexing word vectors.
Found 400000 word vectors.


In [104]:
tokenizer = Tokenizer(num_words=10000,char_level=False)
tokenizer.fit_on_texts(X_data)
sequences_train = tokenizer.texts_to_sequences(X_data)
word_index = tokenizer.word_index
print('Found %s unique tokens in Train.' % len(word_index))
data_train1 = pad_sequences(sequences_train, maxlen=64)
print('Shape of Train data tensor:', data_train1.shape)

Found 25876 unique tokens in Train.
Shape of Train data tensor: (12691, 64)


In [105]:
print(len(sequences_train))


12691


In [106]:
data_train1

array([[  46,  694,  322, ..., 1196,  507, 3831],
       [   0,    0,    0, ...,  230,  712,  111],
       [  66,  380,  110, ...,  239, 2487,   96],
       ...,
       [   0,    0,    0, ..., 1441, 3042, 6728],
       [   0,    0,    0, ...,   18,   80,  874],
       [   0,    0,    0, ...,  917,  275,   47]])

In [107]:
MAX_NUM_WORDS=10000
MAX_SEQUENCE_LENGTH=64
## More code adapted from the keras reference (https://github.com/keras-team/keras/blob/master/examples/pretrained_word_embeddings.py)
# prepare embedding matrix 
from keras.layers import Embedding
from keras.initializers import Constant

## EMBEDDING_DIM =  ## seems to need to match the embeddings_index dimension
EMBEDDING_DIM = embeddings_index.get('a').shape[0]
num_words = min(MAX_NUM_WORDS, len(word_index)) + 1
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i > MAX_NUM_WORDS:
        continue
    embedding_vector = embeddings_index.get(word) ## This references the loaded embeddings dictionary
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

# load pre-trained word embeddings into an Embedding layer
# note that we set trainable = False so as to keep the embeddings fixed
embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

In [108]:
#embedding_matrix = np.zeros((len(word_index) + 1, 50))
#for word, i in word_index.items():
#    embedding_vector = embeddings_index.get(word)
#    if embedding_vector is not None:
#        words not found in embedding index will be all-zeros.
#        embedding_matrix[i] = (embedding_vector)

In [109]:
#embedding_matrix.shape

In [110]:
#embedding_matrix[8048898]

In [111]:
def embedding_output(data):
    emb_dim=50
    max_len=64
    emb=np.zeros((data.shape[0],max_len,emb_dim))
    print(emb.shape)
    for i in range(data.shape[0]):
        
        for j in range(64):
            try:
                emb[i][j]=embeddings_index[data[i][j].lower()]
            except:
                emb[i][j]=np.zeros((50,))
        return emb

In [112]:
e=embedding_output(X_data)

(12691, 64, 50)


In [113]:
texts=np.array(X_data)

In [114]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
Y=le.fit_transform(Y)

In [115]:
from keras.utils import to_categorical
labels = to_categorical(np.asarray(Y))

print(labels.shape)

(12691, 6)


In [116]:
## Code from: https://medium.com/@sabber/classifying-yelp-review-comments-using-cnn-lstm-and-pre-trained-glove-word-embeddings-part-3-53fcea9a17fa
## To create and visualize a model

from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation

model = Sequential()

model.add(LSTM(64,input_shape=(64,50),return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64,input_shape=(64,50),return_sequences=False))
model.add(Dropout(0.2))

model.add(Dense(6, activation='relu'))


In [117]:

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [118]:
#hist=model.fit(e,labels,epochs=50,batch_size=64,shuffle=True,validation_split=0.1)

In [119]:
#CREATING A MODEL IN KERAS
from keras.layers import Embedding,Dense,SimpleRNN
from keras.models import Sequential
model=Sequential()
model.add(Embedding(10000,64))      # 10000 is the vocab_size and 64 is the output dimension i.e. k==embedding size
model.add(SimpleRNN(64))           # 32  is the shape of context vector/ activation vector
model.add(Dense(6,activation="sigmoid"))

In [120]:

# compile our model 
model.compile(optimizer="Adam",loss="binary_crossentropy",metrics=["acc"])

In [121]:
model.evaluate(data_train1,labels)

12691/12691 [==============================] - 3s 272us/step


[0.7099638139137318, 0.46393775939941406]

In [122]:

from keras.layers import Layer
import keras.backend as K

In [123]:
class attention(Layer):
    def __init__(self,**kwargs):
        super(attention,self).__init__(**kwargs)

    def build(self,input_shape):
        self.W=self.add_weight(name="att_weight",shape=(input_shape[-1],1),initializer="normal")
        self.b=self.add_weight(name="att_bias",shape=(input_shape[1],1),initializer="zeros")        
        super(attention, self).build(input_shape)

    def call(self,x):
        et=K.squeeze(K.tanh(K.dot(x,self.W)+self.b),axis=-1)
        at=K.softmax(et)
        at=K.expand_dims(at,axis=-1)
        output=x*at
        return K.sum(output,axis=1)

    def compute_output_shape(self,input_shape):
        return (input_shape[0],input_shape[-1])

    def get_config(self):
        return super(attention,self).get_config()

In [124]:
from numpy import newaxis
from keras.layers import Lambda

In [125]:
def axis(a):
    a = a[ :, :, newaxis]
    return a


In [127]:
word_vector_matrix = np.zeros((40000, 50))
unavailable = 0

for word, index in tokenizer.word_index.items():
    vector = embeddings_index.get(word)
    if vector is not None:
        word_vector_matrix[index] = vector
    else:
        unavailable = unavailable + 1
        #print(word)

In [129]:
import keras
sequence_input = keras.layers.Input(shape=(64,), dtype='int32')
x = keras.layers.Embedding(40000, 50, input_length = 64, weights = [word_vector_matrix], trainable = False)(sequence_input)

In [130]:
att_out=attention()(x)

In [131]:
x = Lambda(axis)(att_out)

In [132]:
y = keras.layers.Conv1D(96,(11),strides = 2,activation = 'relu',input_shape=(100,1))(x)
y = keras.layers.MaxPooling1D(3,2)(y)

y = keras.layers.Conv1D(256,5,padding = 'same',activation = 'relu')(y)
y = keras.layers.MaxPooling1D(3,2)(y)

y = keras.layers.Conv1D(384,3,padding='same',activation= 'relu')(y)
y = keras.layers.Conv1D(384,3,padding='same',activation= 'relu')(y)
y = keras.layers.Conv1D(256,3,padding='same',activation= 'relu')(y)
y = keras.layers.MaxPooling1D(3,2)(y)

y = keras.layers.Flatten()(y)
y = keras.layers.Dense(128, activation = 'relu')(y)
y = keras.layers.Dense(256, activation = 'relu')(y)
y = keras.layers.Dense(64, activation = 'relu')(y)


In [133]:

output=keras.layers.Dense(1,activation='sigmoid',trainable=True)(y)

In [134]:
model=keras.Model(inputs = sequence_input,outputs = output)
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 64)                0         
_________________________________________________________________
embedding_8 (Embedding)      (None, 64, 50)            2000000   
_________________________________________________________________
attention_1 (attention)      (None, 50)                114       
_________________________________________________________________
lambda_1 (Lambda)            (None, 50, 1)             0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 20, 96)            1152      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 9, 96)             0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 9, 256)            1231

In [135]:
adam = keras.optimizers.Adadelta(learning_rate=0.0001, rho=0.95)

In [136]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
epochs = 20
batch_size = 25

history = model.fit(data_train1, Y, epochs=epochs, batch_size=batch_size,validation_split = 0.1)

Train on 11421 samples, validate on 1270 samples
Epoch 1/20
11421/11421 [==============================] - 21s 2ms/step - loss: -4825828283071045632.0000 - accuracy: 0.1948 - val_loss: -55385960511479554048.0000 - val_accuracy: 0.2039
Epoch 2/20
11421/11421 [==============================] - 19s 2ms/step - loss: nan - accuracy: 0.0420 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 3/20
11421/11421 [==============================] - 19s 2ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 4/20
11421/11421 [==============================] - 19s 2ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 5/20
 8850/11421 [======================>.......] - ETA: 4s - loss: nan - accuracy: 0.0000e+00

In [31]:
from sklearn.feature_extraction.text import CountVectorizer

In [32]:
cv=CountVectorizer(stop_words="english")

In [33]:
Se=cv.fit_transform(X_data)

In [34]:
print(Se)

  (0, 7155)	2
  (0, 5292)	4
  (0, 22799)	6
  (0, 16109)	5
  (0, 10258)	5
  (0, 24360)	2
  (0, 3360)	2
  (0, 18685)	2
  (0, 10445)	2
  (0, 4218)	4
  (0, 1705)	2
  (0, 9726)	1
  (0, 22441)	1
  (0, 23523)	1
  (0, 20968)	1
  (0, 8031)	1
  (0, 10270)	2
  (0, 10838)	1
  (0, 10401)	1
  (0, 8459)	1
  (0, 24568)	2
  (0, 350)	1
  (0, 5117)	1
  (0, 22598)	1
  (0, 8210)	1
  :	:
  (10053, 19505)	1
  (10053, 21765)	1
  (10053, 13203)	1
  (10053, 14804)	1
  (10053, 1108)	1
  (10053, 6073)	1
  (10053, 25524)	1
  (10053, 23903)	1
  (10053, 25652)	2
  (10053, 5663)	1
  (10053, 17010)	1
  (10053, 10043)	1
  (10053, 8663)	1
  (10053, 17573)	1
  (10053, 16649)	1
  (10053, 13299)	1
  (10053, 17626)	1
  (10053, 24250)	2
  (10053, 1942)	1
  (10053, 18335)	1
  (10053, 23365)	1
  (10053, 5702)	1
  (10053, 14780)	1
  (10053, 12652)	1
  (10053, 13637)	1


In [35]:
Se.shape

(10054, 26831)

In [36]:
print(cv.get_feature_names()[:100])

['00', '000', '000a', '000new', '000peopl', '005', '010', '011', '012', '014', '017', '018', '02', '020', '022', '024', '027', '028', '029', '03', '032', '033', '036', '04', '041', '042', '046', '05', '050', '054th', '055', '06', '060', '07', '070', '071', '072', '075', '076', '077', '079', '08', '081', '083', '088', '09', '090', '091', '092', '095', '0em', '0px', '10', '100', '1000', '100px', '100th', '100the', '101', '102', '1021', '103', '1033', '104', '1040', '105', '106', '107', '1070', '108', '108th', '109', '10932', '1096', '109th', '10px', '10th', '11', '110', '11023', '1104', '110th', '111', '111th', '112', '112th', '113', '113th', '114', '115', '116', '117', '119', '11900', '11and', '11s', '11th', '12', '120', '120px']


In [37]:
vc=Se.toarray()

In [38]:
vc

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [39]:
4 in vc

True

In [40]:
vc.shape

(10054, 26831)

In [41]:
vc=vc[:,:10000]

In [35]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train , y_test = train_test_split(data_train1,Y,test_size = 0.2)

In [43]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train , y_test = train_test_split(vc,Y,test_size = 0.2)

In [36]:
y_train

array([0, 4, 3, ..., 0, 1, 5])

In [37]:
print(y_train.shape,y_test.shape)

(10152,) (2539,)


In [38]:
x_test

array([[   2,   28,   54, ..., 1007,   40, 1679],
       [  14,  608,  519, ...,  608,  456, 1521],
       [   0,    0,    0, ...,  800, 2472, 1946],
       ...,
       [  34,  198,   51, ..., 1186,   70,    3],
       [   0,    0,    0, ...,   77, 4496,  813],
       [   0,    0,    0, ...,  319,   62,   77]])

In [39]:
x_test.shape

(2539, 64)

In [40]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [114]:
tfidf=TfidfVectorizer()

In [116]:
vc=tfidf.fit_transform(A).toarray()

NameError: name 'A' is not defined

In [117]:
tfidf.vocabulary_

AttributeError: 'TfidfVectorizer' object has no attribute 'vocabulary_'

In [41]:
x_train=pd.DataFrame(x_train)

In [42]:
x_train.to_csv("train_data.csv",index=False)

In [43]:
y_train=pd.DataFrame(y_train)

In [44]:
y_train.to_csv("train_data_labels.csv",index=False)

In [45]:
y_train=pd.read_csv("train_data_labels.csv")

In [46]:
y_train.head()

,0
0,0
1,4
2,3
3,0
4,3


In [47]:
y_test=pd.DataFrame(y_test)

In [48]:
y_test.to_csv("y_test_labels.csv",index=False)

In [49]:
x_test

array([[   2,   28,   54, ..., 1007,   40, 1679],
       [  14,  608,  519, ...,  608,  456, 1521],
       [   0,    0,    0, ...,  800, 2472, 1946],
       ...,
       [  34,  198,   51, ..., 1186,   70,    3],
       [   0,    0,    0, ...,   77, 4496,  813],
       [   0,    0,    0, ...,  319,   62,   77]])

In [50]:
x_test=pd.DataFrame(x_test)

In [51]:
x_test.to_csv("data_train_test.csv",index=False)

In [52]:
y_train

,0
0,0
1,4
2,3
3,0
4,3
...,...
10147,1
10148,5
10149,0
10150,1


In [53]:
Y=y_train.values

In [54]:
np.unique(Y)

array([0, 1, 2, 3, 4, 5], dtype=int64)

In [55]:
from sklearn.ensemble import RandomForestClassifier    

In [56]:
y_train.shape

(10152, 1)

In [57]:
y_test.shape

(2539, 1)

In [58]:
rf=RandomForestClassifier()

In [59]:
rf.fit(x_train,y_train)

C:\Users\hp\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


RandomForestClassifier()

In [60]:
rf.score(x_train,y_train)

1.0

In [61]:
rf.predict(x_test)

array([0, 3, 2, ..., 2, 3, 0], dtype=int64)

In [62]:
rf.score(x_test,y_test)

0.2032296179598267